In [ ]:
import torch
torch.cuda.set_device(1)

## Encapsulation of the fine-tuning process
- Preprocess the dataset.
- 

In [1]:
# required initial processing for fine-tuning
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

# call dataset. Call tokenizer from a checkpoint
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# Tokenize the dataset and batch it. Truncating it is more efficient than padding to the max, batching makes it faster for training
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
# Pad the dataset to the max of the batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [3]:
from transformers import TrainingArguments, AutoModelForSequenceClassification

# Define a TrainingArguments class that will contain all the hyperparameters the Trainer will use for training and evaluation. 
# "test-trainer" is the directory we will be storing the trained model and the checkpoints.
training_args = TrainingArguments("test-trainer")

# Define model with two labels
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
import evaluate

# define a compute metric for evaluation of the model
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [12]:
from transformers import Trainer

# And to see it used in action to report metrics at the end of each epoch, 
# here is how we define a new Trainer with this compute_metrics() function:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# This will report the validation loss and metrics at the end of each epoch on top of the training loss
trainer.train()

/Users/antoine/miniconda3/envs/diffusion/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1377 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

ImportError: To be able to use evaluate-metric/glue, you need to install the following dependencies['sklearn'] using 'pip install sklearn' for instance'

In [5]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [6]:
trainer.train()

/Users/antoine/miniconda3/envs/diffusion/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1377 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.5517, 'learning_rate': 3.184458968772695e-05, 'epoch': 1.09}
{'loss': 0.314, 'learning_rate': 1.3689179375453886e-05, 'epoch': 2.18}
{'train_runtime': 1127.8283, 'train_samples_per_second': 9.757, 'train_steps_per_second': 1.221, 'train_loss': 0.35434758083146817, 'epoch': 3.0}


TrainOutput(global_step=1377, training_loss=0.35434758083146817, metrics={'train_runtime': 1127.8283, 'train_samples_per_second': 9.757, 'train_steps_per_second': 1.221, 'train_loss': 0.35434758083146817, 'epoch': 3.0})

Evaluation -- we can build a useful compute_metrics() function

In [7]:
# outputs three fields (named tuples): predictions, label_ids, and metrics
# metrics field will just contain the loss on the dataset passed, as well as some time metrics (how long it took to predict, in total and on average).
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

  0%|          | 0/51 [00:00<?, ?it/s]

(408, 2) (408,)


In [8]:
# get the argmax from the logits output
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)